In [1]:
import matplotlib.pyplot as plt
import pandas as pd
import numpy as np
import requests
import time
from scipy.stats import linregress
import math

In [2]:
anime = pd.read_csv("data/anime.csv")
anime.head()

,title,mediaType,eps,duration,ongoing,startYr,finishYr,sznOfRelease,description,studios,tags,contentWarn,watched,watching,wantWatch,dropped,rating,votes
0,Fullmetal Alchemist: Brotherhood,TV,64.0,NaN,False,2009.0,2010.0,Spring,The foundation of alchemy is based on the law ...,['Bones'],"['Action', 'Adventure', 'Drama', 'Fantasy', 'M...","['Animal Abuse', 'Mature Themes', 'Violence', ...",103707.0,14351,25810,2656,4.702,86547.0
1,your name.,Movie,1.0,107.0,False,2016.0,2016.0,NaN,Mitsuha and Taki are two total strangers livin...,['CoMix Wave Films'],"['Drama', 'Romance', 'Body Swapping', 'Gender ...",[],58831.0,1453,21733,124,4.663,43960.0
2,A Silent Voice,Movie,1.0,130.0,False,2016.0,2016.0,NaN,"After transferring into a new school, a deaf g...",['Kyoto Animation'],"['Drama', 'Shounen', 'Disability', 'Melancholy...","['Bullying', 'Mature Themes', 'Suicide']",45892.0,946,17148,132,4.661,33752.0
3,Haikyuu!! Karasuno High School vs Shiratorizaw...,TV,10.0,NaN,False,2016.0,2016.0,Fall,"Picking up where the second season ended, the ...",['Production I.G'],"['Shounen', 'Sports', 'Animeism', 'School Club...",[],25134.0,2183,8082,167,4.660,17422.0
4,Attack on Titan 3rd Season: Part II,TV,10.0,NaN,False,2019.0,2019.0,Spring,The battle to retake Wall Maria begins now! Wi...,['Wit Studio'],"['Action', 'Fantasy', 'Horror', 'Shounen', 'Da...","['Cannibalism', 'Explicit Violence']",21308.0,3217,7864,174,4.650,15789.0


In [3]:
#Clean studios column from "list of strings" to a string
anime['studios'] = anime['studios'].str.replace("[", "")
anime['studios'] = anime['studios'].str.replace("]", "")
anime['studios'] = anime['studios'].str.lstrip("'").str.rstrip("'")
#final_username = username.replace("_", "")


In [4]:
#Clean list function
def clean_list(col):
    col = col.str.replace("[", "")
    col = col.str.replace("]", "")
    col = col.str.replace("'", "")
    col = col.str.split(",")
    return col

anime['tags'] = clean_list(anime['tags'])
anime['contentWarn'] = clean_list(anime['contentWarn'])



# anime['tags'] = anime['tags'].str.replace("[", "")
# anime['tags'] = anime['tags'].str.replace("]", "")
# anime['tags'] = anime['tags'].str.replace("'", "")
# anime['tags'] = anime['tags'].str.split(",")

In [5]:
#Strip off the space in front of each word/phrase in a column with a list. 
def leftstrip(li):
    newList = []
    for word in li:
        word = word.strip()
        newList.append(word)
#         print(word)
    return newList


        
anime['tags'] = anime['tags'].apply(lambda x: leftstrip(x))
anime['contentWarn'] = anime['contentWarn'].apply(lambda x: leftstrip(x))        

In [6]:
#Change year to int instead of double to remove decimal
anime["startYr"] = anime["startYr"].fillna(0.0).astype(int)
anime["finishYr"] = anime["finishYr"].fillna(0.0).astype(int)
anime.head()

,title,mediaType,eps,duration,ongoing,startYr,finishYr,sznOfRelease,description,studios,tags,contentWarn,watched,watching,wantWatch,dropped,rating,votes
0,Fullmetal Alchemist: Brotherhood,TV,64.0,NaN,False,2009,2010,Spring,The foundation of alchemy is based on the law ...,Bones,"[Action, Adventure, Drama, Fantasy, Mystery, S...","[Animal Abuse, Mature Themes, Violence, Domest...",103707.0,14351,25810,2656,4.702,86547.0
1,your name.,Movie,1.0,107.0,False,2016,2016,NaN,Mitsuha and Taki are two total strangers livin...,CoMix Wave Films,"[Drama, Romance, Body Swapping, Gender Bender,...",[],58831.0,1453,21733,124,4.663,43960.0
2,A Silent Voice,Movie,1.0,130.0,False,2016,2016,NaN,"After transferring into a new school, a deaf g...",Kyoto Animation,"[Drama, Shounen, Disability, Melancholy, Menta...","[Bullying, Mature Themes, Suicide]",45892.0,946,17148,132,4.661,33752.0
3,Haikyuu!! Karasuno High School vs Shiratorizaw...,TV,10.0,NaN,False,2016,2016,Fall,"Picking up where the second season ended, the ...",Production I.G,"[Shounen, Sports, Animeism, School Club, Schoo...",[],25134.0,2183,8082,167,4.660,17422.0
4,Attack on Titan 3rd Season: Part II,TV,10.0,NaN,False,2019,2019,Spring,The battle to retake Wall Maria begins now! Wi...,Wit Studio,"[Action, Fantasy, Horror, Shounen, Dark Fantas...","[Cannibalism, Explicit Violence]",21308.0,3217,7864,174,4.650,15789.0


In [7]:
#Sort the list by rating
anime.sort_values(['rating'], ascending=False)

,title,mediaType,eps,duration,ongoing,startYr,finishYr,sznOfRelease,description,studios,tags,contentWarn,watched,watching,wantWatch,dropped,rating,votes
0,Fullmetal Alchemist: Brotherhood,TV,64.0,NaN,False,2009,2010,Spring,The foundation of alchemy is based on the law ...,Bones,"[Action, Adventure, Drama, Fantasy, Mystery, S...","[Animal Abuse, Mature Themes, Violence, Domest...",103707.0,14351,25810,2656,4.702,86547.0
1,your name.,Movie,1.0,107.0,False,2016,2016,NaN,Mitsuha and Taki are two total strangers livin...,CoMix Wave Films,"[Drama, Romance, Body Swapping, Gender Bender,...",[],58831.0,1453,21733,124,4.663,43960.0
2,A Silent Voice,Movie,1.0,130.0,False,2016,2016,NaN,"After transferring into a new school, a deaf g...",Kyoto Animation,"[Drama, Shounen, Disability, Melancholy, Menta...","[Bullying, Mature Themes, Suicide]",45892.0,946,17148,132,4.661,33752.0
3,Haikyuu!! Karasuno High School vs Shiratorizaw...,TV,10.0,NaN,False,2016,2016,Fall,"Picking up where the second season ended, the ...",Production I.G,"[Shounen, Sports, Animeism, School Club, Schoo...",[],25134.0,2183,8082,167,4.660,17422.0
4,Attack on Titan 3rd Season: Part II,TV,10.0,NaN,False,2019,2019,Spring,The battle to retake Wall Maria begins now! Wi...,Wit Studio,"[Action, Fantasy, Horror, Shounen, Dark Fantas...","[Cannibalism, Explicit Violence]",21308.0,3217,7864,174,4.650,15789.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
14573,"Welcome to Demon School, Iruma-kun 2",TV,NaN,NaN,False,2021,2021,NaN,"Second Season of Welcome to Demon School, Irum...",,"[Comedy, Shounen, Demons, Monster School, Pers...",[],0.0,0,1106,0,NaN,NaN
14574,Pinocchio-P: Sekai wa Mada Hajimatte Sura Inai,Music Video,1.0,4.0,False,2020,2020,NaN,NaN,,"[Chibi, Vocaloid]",[],9.0,1,7,1,NaN,NaN
14575,Minagoroshi,OVA,1.0,1.0,False,2005,2005,NaN,NaN,,"[Comedy, Ecchi, No Dialogue, Shorts]",[],3.0,0,7,0,NaN,NaN
14576,Kurayukaba,Movie,NaN,NaN,False,0,0,NaN,NaN,Makaria,[],[],0.0,0,62,0,NaN,NaN


In [8]:
anime['mediaType'].value_counts()

TV             4510
Movie          2512
OVA            2097
Music Video    1596
Web            1555
DVD Special     849
Other           782
TV Special      609
Name: mediaType, dtype: int64

In [9]:
#Create different dataframes depending on their type. Created only three most popular
tv_anime = anime.loc[anime['mediaType'] == 'TV'].sort_values('rating', ascending=False)
movie_anime = anime.loc[anime['mediaType'] == 'Movie'].sort_values('rating', ascending=False)
ova_anime = anime.loc[anime['mediaType'] == 'OVA'].sort_values('rating', ascending=False)

In [10]:
tv_anime['sznOfRelease'].value_counts()

Spring    1099
Fall       957
Winter     665
Summer     630
Name: sznOfRelease, dtype: int64

In [11]:
tv_anime['sznOfRelease'].head(100).value_counts()

Spring    31
Fall      30
Winter    23
Summer    16
Name: sznOfRelease, dtype: int64

In [12]:
# count = 0
# for index, col in tv_anime['tags'].iteritems():
#     if 'Action' in col:
#         print(col)
#         print()
#         count +=1
# count    

#Create a function that can count the number per genre
def genreIndex(genre, df):
    count = 0
    indeces = []
    for index, col in df['tags'].iteritems():
        if genre in col:
            indeces.append(index)
#             print(col)
#             print()
            count+=1
    print("There are " + str(count) + " animes")
    return indeces

#Can get total count by getting len(genreIndex)

In [13]:
genreIndex('Romance', tv_anime)
# for index in genreIndex('Romance', tv_anime):
#     print(tv_anime.iloc[index])
# #tv_anime.iloc[0]


There are 544 animes


[13,
 26,
 70,
 75,
 82,
 101,
 123,
 127,
 128,
 132,
 140,
 147,
 148,
 158,
 163,
 166,
 178,
 192,
 202,
 223,
 226,
 228,
 238,
 243,
 245,
 264,
 282,
 299,
 308,
 310,
 319,
 348,
 355,
 350,
 365,
 374,
 375,
 381,
 383,
 391,
 395,
 397,
 405,
 407,
 424,
 430,
 433,
 436,
 440,
 453,
 454,
 485,
 518,
 521,
 530,
 549,
 567,
 568,
 600,
 625,
 646,
 651,
 671,
 688,
 686,
 691,
 698,
 710,
 726,
 727,
 749,
 756,
 762,
 765,
 771,
 777,
 782,
 784,
 809,
 825,
 833,
 869,
 871,
 883,
 885,
 888,
 894,
 909,
 910,
 911,
 913,
 915,
 924,
 927,
 947,
 930,
 966,
 969,
 984,
 993,
 1003,
 1030,
 1041,
 1042,
 1053,
 1060,
 1062,
 1070,
 1079,
 1092,
 1101,
 1110,
 1122,
 1136,
 1141,
 1166,
 1177,
 1168,
 1190,
 1198,
 1226,
 1236,
 1233,
 1258,
 1268,
 1291,
 1294,
 1300,
 1330,
 1341,
 1351,
 1371,
 1372,
 1374,
 1379,
 1389,
 1415,
 1419,
 1423,
 1430,
 1433,
 1466,
 1462,
 1492,
 1504,
 1506,
 1518,
 1533,
 1540,
 1555,
 1549,
 1556,
 1564,
 1586,
 1609,
 1634,
 1646,
 1660,

In [14]:
#Get all the unique genres and the number of animes per genre
genres = []
genreTotal = {}
for index, col in tv_anime['tags'].iteritems():
    for genre in col:
        if (genre not in genres):
            genres.append(genre)
            genreTotal[genre] = 1

        genreTotal[genre] += 1           
print(genreTotal)

{'Action': 1297, 'Adventure': 694, 'Drama': 636, 'Fantasy': 968, 'Mystery': 169, 'Shounen': 785, 'Conspiracy': 37, 'Military': 131, 'Siblings': 78, 'Based on a Manga': 1655, 'Sports': 226, 'Animeism': 49, 'School Club': 160, 'School Life': 699, 'Tournaments': 135, 'Volleyball': 11, 'Horror': 103, 'Dark Fantasy': 39, 'Isolated Society': 14, 'Outside World': 12, 'Post-apocalyptic': 82, 'Comedy': 1669, 'Demons': 142, 'Historical': 193, 'Martial Arts': 72, 'Orphans': 50, 'Monsters': 122, 'Superpowers': 237, 'Sci Fi': 872, 'Aliens': 155, 'Breaking the Fourth Wall': 38, 'Crude': 48, 'Feudal Japan': 81, 'Gag': 126, 'Parody': 139, 'Samurai': 63, 'Slapstick': 53, 'Swordplay': 79, 'Psychic Powers': 41, 'Psychological': 106, 'Supernatural': 398, 'LGBT Themes': 34, 'Thriller': 35, 'Time Travel': 52, 'Based on a Visual Novel': 152, 'Romance': 545, 'Seinen': 479, 'Episodic': 176, 'Love Confession': 6, 'Mind Games': 14, 'Student Council': 15, 'Mecha': 315, 'Overpowered Main Characters': 85, 'Politica

In [15]:
print(len(genres))
print(len(genreTotal))

473
473


In [16]:
#Get the top 20 genres and the total count of anime tv shows
genre_df = pd.DataFrame(list(genreTotal.items()), columns=['Genre', 'Count'])
top20 = genre_df.sort_values('Count', ascending=False).head(20)
top20.reset_index(drop=True, inplace=True)
top20

,Genre,Count
0,Comedy,1669
1,Based on a Manga,1655
2,Action,1297
3,Fantasy,968
4,Sci Fi,872
5,Shounen,785
6,Original Work,730
7,Short Episodes,724
8,Family Friendly,706
9,School Life,699


In [17]:
#Find average rating per genre
#Note: totalAnime is only for those with a score. Does not include the animes w/ no score
def averagePerGenre(df, genre):
    averageScore = 0
    totalScore = 0
    totalAnime = 0
    for index, data in df.iterrows():
        if (genre in data['tags']) & (math.isnan(data['rating']) == False):
            #print(data['rating'])
            totalScore += float(data['rating'])
            totalAnime += 1
#     print(genre)
#     print("Total Score " + str(totalScore))
#     print("Total Anime " + str(totalAnime))
#     print("Average Score " + str(totalScore/totalAnime))
#     print()
    return (totalScore/totalAnime)

print(averagePerGenre(tv_anime, 'Action'))

top20['Average_Score'] = top20['Genre'].apply(lambda x: averagePerGenre(tv_anime, x))

3.340964374482191


In [18]:
top20

,Genre,Count,Average_Score
0,Comedy,1669,3.179969
1,Based on a Manga,1655,3.469324
2,Action,1297,3.340964
3,Fantasy,968,3.296306
4,Sci Fi,872,3.208030
5,Shounen,785,3.526268
6,Original Work,730,3.058659
7,Short Episodes,724,2.404506
8,Family Friendly,706,2.714965
9,School Life,699,3.475290
